<a href="https://colab.research.google.com/github/vellamike/pycuda/blob/master/pycuda_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to CUDA and PyCUDA

In [54]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [55]:
import numpy
numpy.random.seed(1729)
a = numpy.random.randn(4,4)

In [56]:
a = a.astype(numpy.float32)

In [57]:
a_gpu = cuda.mem_alloc(a.nbytes)

In [58]:
cuda.memcpy_htod(a_gpu, a)


In [59]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

In [60]:
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

In [61]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled)
print(a)

[[-1.3746789  -1.6419895   3.3047218  -1.1505861 ]
 [ 2.1979356   1.8518921  -1.9868276  -1.7164422 ]
 [ 0.14977352  1.058711    0.2419031  -0.44884723]
 [-3.113357    0.11188176  0.32294306 -4.2692833 ]]
[[-0.6873394  -0.82099473  1.6523609  -0.57529306]
 [ 1.0989678   0.92594606 -0.9934138  -0.8582211 ]
 [ 0.07488676  0.5293555   0.12095155 -0.22442362]
 [-1.5566785   0.05594088  0.16147153 -2.1346416 ]]


In [62]:
b = numpy.random.randn(4,4)
b = b.astype(numpy.float32)
c = numpy.random.randn(4,4)
c = c.astype(numpy.float32)

print(b)
print(c)

[[ 0.10967004  0.44301215  0.39626622  0.2497974 ]
 [ 1.2984973  -1.2804337  -0.97546583 -0.26908663]
 [-1.1057384  -0.1279927  -0.61782736 -0.98912627]
 [-2.8598924  -0.7943475  -0.30579695  1.7006376 ]]
[[-0.63544416  0.00457387  1.133304    0.14164127]
 [ 2.091718   -1.2695593  -0.7228234  -0.01938889]
 [-0.18282357 -1.1190658  -0.6982751   2.1070845 ]
 [-0.0025556   0.40901005  0.28363675 -1.9498545 ]]


In [63]:
mod2 = SourceModule("""
  __global__ void add2(float *a, float *b)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] += b[idx];
  }
  """)

In [64]:
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)


In [65]:
func = mod2.get_function("add2")
func(b_gpu,c_gpu, block=(4,4,1))

In [66]:
added = numpy.empty_like(b)
cuda.memcpy_dtoh(added, b_gpu)
print(added)
print(b)
print(c)

[[-0.5257741   0.447586    1.5295702   0.39143866]
 [ 3.3902154  -2.549993   -1.6982892  -0.2884755 ]
 [-1.2885619  -1.2470585  -1.3161025   1.1179583 ]
 [-2.862448   -0.38533747 -0.0221602  -0.24921691]]
[[ 0.10967004  0.44301215  0.39626622  0.2497974 ]
 [ 1.2984973  -1.2804337  -0.97546583 -0.26908663]
 [-1.1057384  -0.1279927  -0.61782736 -0.98912627]
 [-2.8598924  -0.7943475  -0.30579695  1.7006376 ]]
[[-0.63544416  0.00457387  1.133304    0.14164127]
 [ 2.091718   -1.2695593  -0.7228234  -0.01938889]
 [-0.18282357 -1.1190658  -0.6982751   2.1070845 ]
 [-0.0025556   0.40901005  0.28363675 -1.9498545 ]]


# Exercises

1. Write a cuda kernel to find the elementwise square of a matrix
2. Write a cuda kernel to find a matrix, which when added to the given matrix results in every element being equal to zero
3. Write a cuda kernel to multiply two matrices - how does it scale with matrix size?

**Question 1**  

In [113]:
mod3 = SourceModule("""
  __global__ void square2(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= a[idx];
  }
  """)

In [114]:
numpy.random.seed(945)
d = numpy.random.randn(4,4)
d = d.astype(numpy.float32)

In [115]:
d_gpu = cuda.mem_alloc(d.nbytes)

In [116]:
cuda.memcpy_htod(d_gpu, d)

In [117]:
func = mod3.get_function("square2")
func(d_gpu, block=(4,4,1))

In [118]:
squared = numpy.empty_like(d)
cuda.memcpy_dtoh(squared, d_gpu)

In [119]:
print(d)
print(squared)

[[ 1.5075933  -3.1299129   0.6720852  -0.26320508]
 [-1.5865606  -0.9187367  -0.16894463 -0.24052033]
 [ 0.43718964  0.90926296 -2.130222    0.33575165]
 [ 0.21738091  0.4459201  -0.17891383  1.7314835 ]]
[[2.2728374  9.796354   0.45169854 0.06927691]
 [2.5171745  0.8440771  0.02854229 0.05785003]
 [0.19113478 0.8267591  4.537846   0.11272917]
 [0.04725446 0.19884475 0.03201016 2.998035  ]]


**Question 2** 

In [106]:
mod4 = SourceModule("""
  __global__ void complement2(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] = - a[idx];
  }
  """)

In [107]:
numpy.random.seed(945)
m = numpy.random.randn(4,4)
m = m.astype(numpy.float32)

In [108]:
m_gpu = cuda.mem_alloc(m.nbytes)

In [109]:
cuda.memcpy_htod(m_gpu, m)

In [110]:
func = mod4.get_function("complement2")
func(m_gpu, block=(4,4,1))

In [111]:
complement = numpy.empty_like(m)
cuda.memcpy_dtoh(complement, m_gpu)

In [112]:
print(m)
print(complement)

[[ 1.5075933  -3.1299129   0.6720852  -0.26320508]
 [-1.5865606  -0.9187367  -0.16894463 -0.24052033]
 [ 0.43718964  0.90926296 -2.130222    0.33575165]
 [ 0.21738091  0.4459201  -0.17891383  1.7314835 ]]
[[-1.5075933   3.1299129  -0.6720852   0.26320508]
 [ 1.5865606   0.9187367   0.16894463  0.24052033]
 [-0.43718964 -0.90926296  2.130222   -0.33575165]
 [-0.21738091 -0.4459201   0.17891383 -1.7314835 ]]


**Question 3**

In [81]:
mod5 = SourceModule("""
  #define N 128
 
  __global__ void multiply(float *a, float *b, float *c)
  {
    
    int row = threadIdx.x + blockIdx.x * blockDim.x;
    int col = threadIdx.y + blockIdx.y * blockDim.y;
    
    if (row < N && col < N){
        c[row * N + col] = 0;
        for(int i=0; i<N; ++i) c[row * N + col] += a[row * N + i] * b[N * i + col];
    }
    
  }
  """)

In [82]:
n = 128

numpy.random.seed(4356)
m_a = numpy.random.randn(n,n)
m_a = m_a.astype(numpy.float32)
m_b = numpy.random.randn(n,n)
m_b = m_b.astype(numpy.float32)
m_c = numpy.empty([n,n])
m_c = m_c.astype(numpy.float32)

In [83]:
ma_gpu = cuda.mem_alloc(m_a.nbytes)
mb_gpu = cuda.mem_alloc(m_b.nbytes)
mc_gpu = cuda.mem_alloc(m_c.nbytes)

In [84]:
cuda.memcpy_htod(ma_gpu, m_a)
cuda.memcpy_htod(mb_gpu, m_b)

In [85]:
block_x = 16
block_y = 16

grid_x = (n + block_x - 1) // block_x
grid_y = (n + block_y - 1) // block_y

In [86]:
from time import perf_counter
from datetime import timedelta

func = mod5.get_function("multiply")

t0 = perf_counter()
func(ma_gpu, mb_gpu, mc_gpu, block=(block_x, block_y, 1), grid = (grid_x, grid_y, 1))
duration = perf_counter() - t0

print("> duration: %s\n" % timedelta(seconds=duration))

> duration: 0:00:00.000144



In [87]:
t0 = perf_counter()
func(ma_gpu, mb_gpu, mc_gpu, block=(block_x, block_y, 1), grid = (grid_x, grid_y, 1))
cuda.Context.synchronize()
duration = perf_counter() - t0

print("> duration: %s\n" % timedelta(seconds=duration))

> duration: 0:00:00.000313



In [88]:
cuda.memcpy_dtoh(m_c, mc_gpu)

In [89]:
print (numpy.matmul(m_a, m_b))
print(m_c)

[[ -3.7378654   4.235635  -17.992908  ...   8.211486   -3.59588
   -4.3585978]
 [  7.2819767   1.769602   -3.9131672 ... -10.885104    9.402001
   -2.3365288]
 [ -4.3908973  -9.043757  -19.409258  ...  13.265595  -13.009256
   -3.158233 ]
 ...
 [ 14.096389  -12.489891    9.665953  ...  10.440536   -6.866548
   22.28903  ]
 [  9.263469   -4.4498777 -22.920935  ...   1.654692    8.994262
   -3.5488815]
 [ -7.881237    8.017286   -8.048629  ... -15.7865     13.421714
    1.4095254]]
[[ -3.7378654   4.235635  -17.992908  ...   8.211486   -3.59588
   -4.3585978]
 [  7.2819767   1.769602   -3.9131672 ... -10.885104    9.402001
   -2.3365288]
 [ -4.3908973  -9.043757  -19.409258  ...  13.265595  -13.009256
   -3.158233 ]
 ...
 [ 14.096389  -12.489891    9.665953  ...  10.440536   -6.866548
   22.28903  ]
 [  9.263469   -4.4498777 -22.920935  ...   1.654692    8.994262
   -3.5488815]
 [ -7.881237    8.017286   -8.048629  ... -15.7865     13.421714
    1.4095254]]


**Question 4**

In [128]:
mod6 = SourceModule("""
  #define N 4096
 
  __global__ void product(float *a, float *b, float *c)
  {
    
    int row = threadIdx.x + blockIdx.x * blockDim.x;
    int col = threadIdx.y + blockIdx.y * blockDim.y;
    
    if (row < N && col < N){
        c[row * N + col] = 0;
        for(int i=0; i<N; ++i) c[row * N + col] += a[row * N + i] * b[N * col + i];
    }
    
  }
  """)

In [129]:
n = 4096

numpy.random.seed(4356)
m_a = numpy.random.randn(n,n)
m_a = m_a.astype(numpy.float32)
m_b = numpy.random.randn(n,n)
m_b = m_b.astype(numpy.float32)
m_c = numpy.empty([n,n])
m_c = m_c.astype(numpy.float32)
m_d = numpy.empty([n,n])
m_d = m_c.astype(numpy.float32)

In [130]:
ma_gpu = cuda.mem_alloc(m_a.nbytes)
mb_gpu = cuda.mem_alloc(m_b.nbytes)
mc_gpu = cuda.mem_alloc(m_c.nbytes)
md_gpu = cuda.mem_alloc(m_d.nbytes)

In [131]:
block_x = 16
block_y = 16

grid_x = (n + block_x - 1) // block_x
grid_y = (n + block_y - 1) // block_y

In [132]:
func = mod6.get_function("product")

In [133]:
from time import perf_counter
from datetime import timedelta

cuda.memcpy_htod(ma_gpu, m_a)
cuda.memcpy_htod(mb_gpu, m_b)

t0 = perf_counter()

func(ma_gpu, mb_gpu, mc_gpu, block=(block_x, block_y, 1), grid = (grid_x, grid_y, 1))
cuda.memcpy_dtoh(m_c, mc_gpu)
func(mb_gpu, ma_gpu, md_gpu, block=(block_x, block_y, 1), grid = (grid_x, grid_y, 1))
cuda.memcpy_dtoh(m_d, md_gpu)
cuda.Context.synchronize()
duration = perf_counter() - t0

print("> duration: %s\n" % timedelta(seconds=duration))

> duration: 0:00:02.754468



In [134]:
t0 = perf_counter()

func(ma_gpu, mb_gpu, mc_gpu, block=(block_x, block_y, 1), grid = (grid_x, grid_y, 1))
cuda.memcpy_dtoh_async(m_c, mc_gpu)
func(mb_gpu, ma_gpu, md_gpu, block=(block_x, block_y, 1), grid = (grid_x, grid_y, 1))
cuda.memcpy_dtoh_async(m_d, md_gpu)
cuda.Context.synchronize()
duration = perf_counter() - t0

print("> duration: %s\n" % timedelta(seconds=duration))

> duration: 0:00:02.714176



**Question 5** 

In [172]:
mod7 = SourceModule("""
  #define N 2048
 
  __global__ void multiply(float *a, float *b, float *c)
  {
    
    for(int row = threadIdx.x + blockIdx.x * blockDim.x; row < N; row += gridDim.x * blockDim.x){
        for(int col = threadIdx.y + blockIdx.y * blockDim.y; col < N; col += gridDim.y * blockDim.y){
            c[row * N + col] = 0;
            for(int i=0; i<N; ++i) c[row * N + col] += a[row * N + i] * b[N * i + col];
        }
    }
    
  }
  """)

In [173]:
n = 2048

numpy.random.seed(4356)
m_a = numpy.random.randn(n,n)
m_a = m_a.astype(numpy.float32)
m_b = numpy.random.randn(n,n)
m_b = m_b.astype(numpy.float32)
m_c = numpy.empty([n,n])
m_c = m_c.astype(numpy.float32)
m_d = numpy.empty([n,n])
m_d = m_c.astype(numpy.float32)

In [174]:
ma_gpu = cuda.mem_alloc(m_a.nbytes)
mb_gpu = cuda.mem_alloc(m_b.nbytes)
mc_gpu = cuda.mem_alloc(m_c.nbytes)
md_gpu = cuda.mem_alloc(m_d.nbytes)

In [175]:
cuda.memcpy_htod(ma_gpu, m_a)
cuda.memcpy_htod(mb_gpu, m_b)

In [176]:
block_x = 16
block_y = 16

grid_x = (n + block_x - 1) // block_x
grid_y = (n + block_y - 1) // block_y

In [177]:
from time import perf_counter
from datetime import timedelta

func = mod7.get_function("multiply")

t0 = perf_counter()
func(ma_gpu, mb_gpu, mc_gpu, block=(block_x, block_y, 1), grid = (grid_x, grid_y, 1))
cuda.Context.synchronize()
duration = perf_counter() - t0

print("> duration: %s\n" % timedelta(seconds=duration))

> duration: 0:00:00.179902



In [178]:
t0 = perf_counter()
func(ma_gpu, mb_gpu, md_gpu, block=(block_x, block_y, 1), grid = (8, 8, 1))
cuda.Context.synchronize()
duration = perf_counter() - t0

print("> duration: %s\n" % timedelta(seconds=duration))

> duration: 0:00:00.222918



In [179]:
cuda.memcpy_dtoh(m_c, mc_gpu)
cuda.memcpy_dtoh(m_d, md_gpu)

In [180]:
print(numpy.max(numpy.abs(m_c - m_d)))

0.0
